<a href="https://colab.research.google.com/github/mahdibaghbanzadeh/NLP/blob/main/notebooks/NLP_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install sklearn -q
# !pip install spacy -q
# !python -m spacy download en_core_web_sm

In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import metrics

import string
import spacy
np.random.seed(42)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv("/content/drive/MyDrive/data/toxic_comments/train.csv")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [22]:
data.shape

(159571, 8)

In [42]:
df_sam = data.sample(frac=0.3)
df_sam.shape

(47871, 8)

In [43]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'see', 'something', 'take', 'wherever', 'here', 'how', 'least', 'call', 'the', 'up', 'using', '’m', 'elsewhere', 'otherwise', 'would', 'whenever', 'throughout', 'various', 'might', 'whole', 'herself', 'it', 'forty', 'hereby', 'not', 'we', 'name', 'more', 'towards', 'than', 'besides', 'above', 'some', 'third', 'any', 'across', 'must', 'them', 'had', "'re", 'do', 'whereafter', 'thereby', 'am', 'anyone', 'however', 'will', 'wherein', 'doing', 'whoever', 'out', 'onto', 'upon', 'say', 'after', 'under', 'although', 'per', 'next', 'itself', 'been', 'full', 'this', 'that', '‘m', 'from', 'unless', 'a', 'herein', 'formerly', 'what', "n't", 'five', 'other', 'she', 'then', 'along', 'thence', 'himself', 'less', 'part', 'first', 'sixty', 'seem', 'since', 'amongst', 'both', 'whom', 'anything', 'were', 'eleven', 'seeming', 'does', 'back', 'which', 'during', 'myself', 'off', 'nobody', 'hence', 'whereas', 'rather', 'but', 'three', 'over', 'is', 'at', 'why', 'very', 'mine', 'afterwards', 'many', 'me', '

In [44]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [45]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [46]:
sentence = "I am eating apple ?"
spacy_tokenizer(sentence)

['eat', 'apple']

In [47]:
count_vector = CountVectorizer(tokenizer = spacy_tokenizer)
# tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [48]:
count_vector.fit_transform(["I am eating apple, I like apple","I am playing cricket"]).toarray()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[2, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]])

In [49]:
count_vector.get_feature_names_out()

array(['apple', 'cricket', 'eat', 'like', 'play'], dtype=object)

In [50]:
count_vector.vocabulary_

{'eat': 2, 'apple': 0, 'like': 3, 'play': 4, 'cricket': 1}

In [51]:
from sklearn.model_selection import train_test_split

X = df_sam['comment_text'] # the features we want to analyze
ylabels = df_sam['toxic'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,stratify=ylabels)

In [52]:
print(sum(ylabels)/len(ylabels) *100)
print(sum(y_train)/len(y_train) *100)


9.61751373482902
9.617192396072696


In [53]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [54]:
X_train_vetcors= count_vector.fit_transform(X_train)
X_test_vetcors= count_vector.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [55]:
X_train_vetcors.shape

(38296, 80233)

In [56]:
X_test_vetcors.shape

(9575, 80233)

In [57]:
X_train_vetcors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [58]:
classifier.fit(X_train_vetcors,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [59]:
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9522715404699739
Logistic Regression Precision: 0.8636363636363636
Logistic Regression Recall: 0.5982627578718784


In [60]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
X_train_vetcors= tfidf_vector.fit_transform(X_train)
X_test_vetcors= tfidf_vector.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [61]:
classifier = LogisticRegression()
classifier.fit(X_train_vetcors,y_train)
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9498694516971279
Logistic Regression Precision: 0.9418837675350702
Logistic Regression Recall: 0.510314875135722
